<a href="https://colab.research.google.com/github/SanjanaRamoliya27/blip-weather-vqa/blob/main/App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Create project folder
!mkdir -p /content/image_project
%cd /content/image_project


/content/image_project


In [2]:
!pip install gradio opencv-python pillow imagehash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 8.7 MB/s eta 0:00:00


In [3]:
%%writefile utils.py

import cv2
import numpy as np
from PIL import Image
import imagehash

def image_basic_info(image):
    width, height = image.size
    aspect_ratio = round(width / height, 2)
    return width, height, aspect_ratio

def blur_score(image):
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    score = cv2.Laplacian(gray, cv2.CV_64F).var()
    return round(score, 2)

def perceptual_hash(image):
    return str(imagehash.phash(image))


Writing utils.py


In [4]:
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-05, e

In [5]:
import gradio as gr
from PIL import Image

from utils import image_basic_info, blur_score


In [6]:
def analyze_image_with_vqa(img, question):
    if img is None or question.strip() == "":
        return "⚠️ Please upload an image and enter a question."

    # -------- EDA --------
    width, height, aspect_ratio = image_basic_info(img)
    blur = blur_score(img)

    quality = "Good"
    if blur < 100:
        quality = "Low Quality (Blurry)"

    # -------- VQA --------
    inputs = processor(img, question, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(**inputs)

    answer = processor.decode(output[0], skip_special_tokens=True)

    result = f"""
🖼️ IMAGE EDA
-------------------------
Width           : {width}
Height          : {height}
Aspect Ratio    : {aspect_ratio}
Blur Score      : {blur}
Image Quality   : {quality}

🤖 VISUAL QUESTION ANSWERING
-------------------------
Question : {question}
Answer   : {answer}
"""

    return result


In [7]:
with gr.Blocks(title="Image EDA & Visual Question Answering (VQA)") as demo:

    gr.Markdown(
        """
        # 🖼️ Image EDA & Visual Question Answering (VQA)
        Analyze image quality and ask questions using the BLIP VQA model.
        """
    )

    with gr.Row():

        # ---------- LEFT COLUMN ----------
        with gr.Column(scale=1):
            image_input = gr.Image(
                type="pil",
                label="Upload Image"
            )

            question_input = gr.Textbox(
                label="Ask a Question about the Image",
                placeholder="What is happening in the image?"
            )

            submit_btn = gr.Button("Submit", variant="primary")
            clear_btn = gr.Button("Clear")

        # ---------- RIGHT COLUMN ----------
        with gr.Column(scale=1):
            output_box = gr.Textbox(
                label="EDA + VQA Output",
                lines=20,
                interactive=False
            )

    # Button actions
    submit_btn.click(
        fn=analyze_image_with_vqa,
        inputs=[image_input, question_input],
        outputs=output_box
    )

    clear_btn.click(
        fn=lambda: ("", None, ""),
        inputs=[],
        outputs=[question_input, image_input, output_box]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3f1eb9241e3c74ddb1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
